# ADMM

* draft 1

* date: 2018/10/16

### Loadmap: 

This file contains the following functions and associated examples: 

* L_num: The function to generate $l = (l_1, l_2)$, where $l_1 < l_2$

* Sylvester equation 

* update_A: function to update A

* prox: function to calculate proximal map 

* update_vz: function for update V and Z

* update_lambda: function for update $lambda_1$ and $lambda_2$

* Bi_ADMM: final admm function

### 1. L_num

The function to generate $l = (l_1, l_2)$, where $l_1 < l_2$

In [1]:
L_num=function(n){
  library(tidyr)
  L=matrix(0,n,n)
  for(i in 1:n){
    for(j in 1:n){
      L[i,j] = paste(i,',',j)
    }
  }
  l=c(L[upper.tri(L)])
  ll=data.frame(l)
  
  LL=ll %>% separate(l,c('l1','l2'),sep=',')
  LL=data.frame(l1=as.numeric(LL[,'l1']),l2=as.numeric(LL[,'l2']))
  return(LL)
}

### 2. Sylvester equation

In [2]:
sylvester = function(A, B, C, eps = 0.0001){
   
    library(MASS)
    library(Matrix)
      
    A1 = Schur(A)
    Q1 = A1$Q; R1 = A1$T
  
    A2 = Schur(B)
    Q2 = A2$Q; R2 = A2$T 
    C = t(Q1) %*% C %*% Q2
  
    Rsq = R1 * R1
    I = diag(dim(A)[1])
  
    k = 1
    n = dim(R2)[1]
    
    while(k < n + 1){
    if(k < n){
      if(abs(R2[k+1, k]) < eps){
        left = R1 + R2[k,k] * I
        right = C[,k]
        temp = matrix(0, dim(X)[1],1)
        if(k == 1){
          temp = temp
        }else{
          for(i in 1:(k-1)){
            temp = temp + X[,i] * R2[i,k]
          }
        }
        temp = matrix(temp, dim(C)[1],1)
        X[,k] = ginv(left) %*% (right - temp)
        mytry = myTryCatch(solve(left))
        if(is.null(mytry$error) == 0){er = c(er,tt)}
        k = k+1
      }else{
        r11 = R2[k,k]; r12 = R2[k, k+1]; r21 = R2[k+1, k]; r22 = R2[k+1, k+1]
        temp2 = matrix(0, dim(X)[1],1);temp3=matrix(0, dim(X)[1],1)
        if(k == 1){
          temp2 = temp2
          temp3 = temp3
        }else{
          for(i in 1:(k-1)){
            temp2 = temp2 + X[,i] * R2[i,k]
            temp3 = temp3 + X[,i] * R2[i,k+1]}
        }
        temp2 = matrix(temp2, dim(X)[1],1)
        temp3 = matrix(temp3, dim(X)[1],1)
        b1 = C[,k] - temp2 
        b2 = C[,k+1] - temp3
        b1_prime = R1 %*% b1 + r22 * b1 - r21 * b2
        b2_prime = R1 %*% b2 + r11 * b2 - r12 * b1
        b_prime = matrix(0, dim(X)[1],2)
        b_prime[,1] = b1_prime; b_prime[,2] = b2_prime
        X[,k:(k+1)] = ginv(R1 %*% R1 + (r11 + r22) * R1 +
                             (r11*r22 - r12*r21) * I) %*% b_prime 
        k = k+2
      }
    }else{
      if(abs(R2[1, k]) > eps){
        left = R1 + R2[k,k] * I
        right = C[,k]
        temp = matrix(0, dim(X)[1],1)
        if(k == 1){
          temp = temp
        }else{
          for(i in 1:(k-1)){
            temp = temp + X[,i] * R2[i,k]
          }
        }
        temp = matrix(temp, dim(C)[1],1)
        X[,k] = ginv(left) %*% (right - temp)
        k = k+1
      }else{
        R22 = R2
        R22 = cbind(R2, rep(0,dim(R2)[1]))
        R22 = rbind(R22,rep(0,dim(R2)[1]+1))
        r11 = R22[k,k]; r12 = R22[k, k+1]; r21 = R22[k+1, k]; r22 = R22[k+1, k+1]
        temp2 = matrix(0, dim(X)[1],1);temp3=matrix(0, dim(X)[1],1)
        for(i in 1:(k-1)){
          temp2 = temp2 + X[,i] * R22[i,k]
          temp3 = temp3 + X[,i] * R22[i,k+1]}
        temp2 = matrix(temp2, dim(X)[1],1)
        temp3 = matrix(temp3, dim(X)[1],1)
        b1 = C[,k] - temp2 
        b2 = - temp3
        b1_prime = R1 %*% b1 + r22 * b1 - r21 * b2
        b2_prime = R1 %*% b2 + r11 * b2 - r12 * b1
        b_prime = matrix(0, dim(X)[1],2)
        b_prime[,1] = b1_prime; b_prime[,2] = b2_prime
        GOD = ginv(R1 %*% R1 + (r11 + r22) * R1 +
                     (r11*r22 - r12*r21) * I) %*% b_prime 
        X[,k] = GOD[,1]
        k = k+2
      }
    }
  }
  return(Q1 %*% X %*% t(Q2))
}

# 3. Update A

To update A, we need to calculate the:

$$AM + NA = C$$

Where:

* M: $I_n + \nu_1 \sum_{k \in \epsilon_1} (e_{l_1} - e_{l_2})(e_{l_1} - e_{2_2})^T$
  
  + $I_n$ is (n,n), $e_l$ is (n,n), M is (n,n) matrix


* N: $\nu_2 \sum_{l \in \epsilon_2}(e_{k_1} - e_{k_2})(e_{k_1} - e_{k_2})^T$
  
  + $e_k$ is (p,p), N is (p,p) matrix


* C: $X + \sum_{l \in \epsilon_1} (e_{l1} - e_{l_2})(\lambda_{l1} + \nu_1 v_l)^T + 
\sum_{k \in \epsilon_2} (\lambda_{k2} + \nu_2 z_k)(e_{k1} - e_{k_2})^T$


In [3]:
library(MASS)
library(Matrix)

In [4]:
update_A = function(X, nu1, nu2, lambda_1, lambda_2, v, z){
  library(MASS)
  library(Matrix)
  n=dim(X)[1]; p=dim(X)[2]
  eplison_n=L_num(n)
  eplison_p=L_num(p)
  
  En=matrix(0,n,n)
  Ep=matrix(0,p,p)
  
  # calculate \sum_{l \in \epsilon_1}(e_{l1} - e_{l_2})(e_{l1} - e_{l_2})^T, which is En
  for(i in 1:dim(eplison_n)[1]){
    l1=eplison_n[i,'l1']
    l2=eplison_n[i,'l2']
    el1=matrix(rep(0,n),n,1)
    el2=matrix(rep(0,n),n,1)
    el1[l1,1]=1;el2[l2,1]=1
    En=En+(el1-el2) %*% t(el1-el2)
  }
  
  # calculate \sum_{k \in \epsilon_2} (e_{k1} - e_{k_2})(e_{k1} - e_{k_2})^T, which is Ep
  for(i in 1:dim(eplison_p)[1]){
    l1=eplison_p[n,'l1']
    l2=eplison_p[n,'l2']
    el1=matrix(rep(0,p),p,1)
    el2=matrix(rep(0,p),p,1)
    el1[l1,1]=1;el2[l2,1]=1
    Ep=Ep+(el1-el2) %*% t(el1-el2)
  }
  
  M = diag(1,n,n) + nu1 * En
  
  N = nu2 * Ep
  
  # C2 is the second part of C, which is \sum_{l \in \epsilon_1} (e_{l1} - e_{l_2})(\lambda_{l1} + \nu_1 v_l)^T 
  C2 = matrix(0,n,p) 
  for(i in 1:dim(eplison_n)[1]){
    l1=eplison_n[i,'l1']
    l2=eplison_n[i,'l2']
    el1=matrix(rep(0,n),n,1)
    el2=matrix(rep(0,n),n,1)
    el1[l1,1]=1;el2[l2,1]=1
    C2 = C2 + (el1-el2) %*% t(lambda_1[,i] + nu1 * v[,i])
  }
  
  # C3 is the third part of C, which is \sum_{k \in \epsilon_2} (\lambda_{k2} + \nu_2 z_k)(e_{k1} - e_{k_2})^T
  C3 = matrix(0,n,p)
  for(i in 1:dim(eplison_p)[1]){
    l1=eplison_p[i,'l1']
    l2=eplison_p[i,'l2']
    el1=matrix(rep(0,p),p,1)
    el2=matrix(rep(0,p),p,1)
    el1[l1,1]=1;el2[l2,1]=1
    C3 = C3+(lambda_2[,i] + nu2 * z[,i]) %*% t(el1-el2)
  }
  
  C = X +  C2 + C3  
  
  A = sylvester(M,N,C)
  
  return(A)
}

### Test whether the function can work 

In [8]:
X = matrix(rnorm(10000),100,100)
n=dim(X)[1]; p=dim(X)[2]
eplison_p=L_num(p)
eplison_n=L_num(n)
nu1=0.001; nu2=0.001
lambda_1 = matrix(1,p,dim(eplison_n)[1])
v = matrix(1,p,dim(eplison_n)[1])
lambda_2 = matrix(1,n,dim(eplison_p)[1])
z = matrix(1,n,dim(eplison_p)[1])

In [9]:
A=update_A(X, nu1, nu2, lambda_1, lambda_2, v, z)

In [10]:
A[1:10,1:10]

188.5100,188.7552,185.1831,183.4560,180.7195,179.6728,178.5762,176.5232,167.3008,170.4056
186.6635,184.7385,182.9870,183.4884,178.8145,178.8810,174.5038,172.8774,166.8182,168.2224
186.9021,184.2826,180.3677,179.0471,177.7816,176.2282,175.5284,170.4522,163.3721,169.7288
183.8371,181.1017,179.3130,176.5568,176.0812,172.5946,172.4373,170.0004,161.8405,166.0663
182.2721,178.9474,177.5913,176.9632,173.8469,172.7803,169.8747,168.3326,160.4031,165.8018
181.8885,177.8572,176.6133,173.9047,172.6792,171.5086,168.0026,167.0089,157.5593,163.1231
178.3806,176.1053,174.4986,171.3763,171.8792,167.8916,168.2031,164.3742,157.1445,159.3589
175.3389,175.5640,172.2470,171.9170,166.5043,166.3545,164.3957,162.0513,155.4987,157.3040
174.8373,171.5222,170.7324,168.0020,165.6907,164.2803,162.0026,161.4708,153.4962,157.2051
171.9407,170.9627,169.9343,167.5116,163.9649,162.6189,160.6231,159.8653,151.3443,153.3174


# 4. Update V and Z

* calculate the proximal mapping following Eric's formula (try $l2$ norm first):

The formula is: 

$$prox_{\sigma \Omega} (v) = [1 - \frac{\sigma}{||v||_2}]_+ v$$

In [11]:
prox = function(v,sigma){
   return(max(1-sigma/sum(v %*% t(v)),0) %*% v)
}

In [12]:
v = 1:5
prox(v, 0.1)

0.9995556,1.999111,2.998667,3.998222,4.997778


In [13]:
dist_weight <- function(X, phi){
  dist_X <- as.numeric(dist(X))
  return(exp(- phi * dist_X^2))
}

In [14]:
tri2vec = function(i,j,n) {
  return(n*(i-1) - i*(i-1)/2 + j -i)
}

In [15]:
knn_weights <- function(w,k,n) {
  i <- 1
  neighbors <- tri2vec(i,(i+1):n,n)
  keep <- neighbors[sort(w[neighbors],decreasing=TRUE,index.return=TRUE)$ix[1:k]]
  for (i in 2:(n-1)) {
    group_A <- tri2vec(i,(i+1):n,n)
    group_B <- tri2vec(1:(i-1),i,n)
    neighbors <- c(group_A,group_B)
    knn <- neighbors[sort(w[neighbors],decreasing=TRUE,index.return=TRUE)$ix[1:k]]
    keep <- union(knn,keep)
  }
  i <- n
  neighbors <- tri2vec(1:(i-1),i,n)
  knn <- neighbors[sort(w[neighbors],decreasing=TRUE,index.return=TRUE)$ix[1:k]]
  keep <- union(knn,keep)
  w[-keep] <- 0
  return(w)
}

In [16]:
update_vz = function(X, A, lambda_1, lambda_2, gamma_1, gamma_2, nu1, nu2){
  
  n=dim(X)[1]; p=dim(X)[2]
  eplison_n=L_num(n)
  eplison_p=L_num(p)
 
  w <- dist_weight(X / sqrt(p),phi = 0.5)
  w_l <- knn_weights(w,k = 5,n)
  
  u = dist_weight(t(X) / sqrt(n),phi = 0.5)
  u_k <- knn_weights(u,k = 5,p)
  
  for(i in 1:dim(eplison_n)[1]){
    l1=eplison_n[i,'l1']
    l2=eplison_n[i,'l2']
    a_l1 = A[l1,]; a_l2 = A[l2,]
    v_temp = a_l1 - a_l2 + 1/nu1 * lambda_1[,i]
    sigma_1l = gamma_1 * w_l[i]/nu1
    v[,i] = prox(v_temp,sigma_1l)
  }
  
  for(i in 1:dim(eplison_p)[1]){
    l1=eplison_p[i,'l1']
    l2=eplison_p[i,'l2']
    a_l1 = A[,l1]; a_l2 = A[,l2]
    v_temp = a_l1 - a_l2 + 1/nu2 * lambda_2[,i]
    #u_k = exp(-0.5 * (t(X[,l1] - X[,l2]) %*% (X[,l1] - X[,l2])))
    sigma_2k = gamma_2 * u_k[i]/nu2
    z[,i] = prox(v_temp,sigma_2k)
  }
  
  return(list(v = v, z = z))
}



# 5. Update $\lambda_1$, $\lambda_2$

$$\lambda_{1l} = \lambda_{1l} + \nu_1 (v_l - \alpha_{l1} + \alpha_{l2})$$

$$\lambda_{k2} = \lambda_{k2} + \nu_2 (z_k - \alpha_{k1} + \alpha_{k2})$$

* $\lambda_{1l}$ is a matrix (p, n(n-1)/2)

* $\lambda_{k2}$ is a matrix (n, p(p-1)/2)

In [17]:
update_lambda=function(X, A, nu1, nu2,v, z){
  n = dim(X)[1]; p = dim(X)[2]
  eplison_p = L_num(p)
  eplison_n = L_num(n)
    # update lambda 1
  for(i in 1:dim(eplison_n)[1]){
    l1=eplison_n[i,'l1']
    l2=eplison_n[i,'l2']
    a_l1 = matrix(A[l1,],p,1)
    a_l2 = matrix(A[l2,],p,1)
    lambda_1[,i] = lambda_1[,i] + nu1 * (v[,i] - a_l1 + a_l2)
  } 
    # update lambda 2
  for(i in 1:dim(eplison_p)[1]){
    l1=eplison_n[i,'l1']
    l2=eplison_n[i,'l2']
    a_k1 = matrix(A[,l1],n,1)
    a_k2 = matrix(A[,l2],n,1)
    lambda_2[,i] = lambda_2[,i] + nu2 * (z[,i] - a_k1 + a_k2)
  }
  return(lambda=list(lambda_1,lambda_2))
}


# ADMM function

Combine previous function together to correct ADMM function

In [18]:
Bi_ADMM = function(X, nu1, nu2, lambda_1, lambda_2, v, z, gamma_1, gamma_2, e = 0.001){
  A = X; 
  for(iter in 1: 100){
    A_old = A; v_old = v; z_old = z; lambda_1_old = lambda_1; lambda_2_old = lambda_2
      
      # update A 
    A = update_A(X, nu1, nu2, lambda_1, lambda_2, v, z)
    
      # update V and Z
    vz = update_vz(X, A, lambda_1, lambda_2, gamma_1, gamma_2, nu1, nu2)
    v = vz[[1]]
    z = vz[[2]]
    
      # update lambda_1 and lambda_2 
    lambda = update_lambda(X, A, nu1, nu2,v, z)
    lambda_1 = lambda[[1]]
    lambda_2 = lambda[[2]]
    
      # whether coverage
    if(sum(abs(A - A_old)) < e & 
       sum(abs(v - v_old)) < e & 
       sum(abs(z - z_old)) < e & 
       sum(abs(lambda_1 - lambda_1_old)) < e & 
       sum(abs(lambda_2 - lambda_2_old)) < e){
      return(list(A = A))
      break
    }
  }
  if(iter == 100){print('not converage within 100 iters')
    return(A)}
}

## Test 

In [19]:
nu1 = 0.001; nu2 = 0.001; 

X = cbind(rnorm(10,3,1), rnorm(10,-3,1),rep(0,20),rep(0,20),rep(0,20),rep(0,20))
X = rbind(X, cbind(rnorm(10,6,1), rnorm(10,-6,1),rep(0,20),rep(0,20),rep(0,20),rep(0,20)))

n = dim(X)[1]; p = dim(X)[2]
eplison_p = L_num(p)
eplison_n = L_num(n)

lambda_1 = matrix(1,p,dim(eplison_n)[1])
v = matrix(1,p,dim(eplison_n)[1])
lambda_2 = matrix(1,n,dim(eplison_p)[1])
z = matrix(1,n,dim(eplison_p)[1])

gamma_1 = 0.1; gamma_2 = 0.1

In [20]:
A = Bi_ADMM(X, nu1, nu2, lambda_1, lambda_2, v, z, gamma_1, gamma_2)

[1] "not converage within 100 iters"


In [21]:
A[1:10,]

8760.772,8353.199,7958.072,7557.694,7157.316,6756.938
8362.535,7956.724,7560.231,7159.853,6759.475,6359.097
7967.780,7558.272,7162.389,6762.011,6361.633,5961.254
7567.331,7160.002,6764.552,6364.174,5963.796,5563.417
7170.470,6762.597,6366.710,5966.332,5565.954,5165.576
6771.696,6366.276,5968.870,5568.492,5168.114,4767.736
6374.066,5969.042,5571.029,5170.651,4770.273,4369.895
5976.838,5570.575,5173.189,4772.811,4372.433,3972.055
5580.434,5172.199,4775.348,4374.970,3974.592,3574.214
5182.771,4776.412,4377.506,3977.128,3576.750,3176.372


In [22]:
X[1:10,]

1.931721,-5.219795,0,0,0,0
1.537857,-3.862220,0,0,0,0
4.607101,-4.467990,0,0,0,0
2.011090,-4.897967,0,0,0,0
2.984927,-4.464296,0,0,0,0
2.057757,-2.954681,0,0,0,0
2.267123,-2.351131,0,0,0,0
2.875783,-2.974508,0,0,0,0
4.303413,-3.506022,0,0,0,0
4.481639,-1.462811,0,0,0,0
